In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = "llama2"
MODEL = "gemma3n:e2b"

In [3]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

model = Ollama(model = MODEL)
embedding = OllamaEmbeddings()
chat = model | parser


/tmp/ipykernel_68449/4230599585.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model = MODEL)
/tmp/ipykernel_68449/4230599585.py:8: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings()


In [4]:
chat.invoke("tell me something funny")

"Why don't scientists trust atoms? \n\n... Because they make up everything! \n\n---\n\nHope that got a chuckle! 😊 \n\nDo you want to hear another one, or maybe a joke about a specific topic? Let me know! 😄\n\n\n\n"

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("jobspecs.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'Skia/PDF m138', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36', 'creationdate': '2025-09-11T11:30:06+00:00', 'title': '(1) LinkedIn', 'moddate': '2025-09-11T11:30:06+00:00', 'source': 'jobspecs.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content="Few&Far\nLead Backend Engineer\n£90K/yr - £110K/yr Hybrid Full-time\nEasy Apply Save\nLondon Area, United Kingdom·3 weeks ago·Over 100 applicants\nPromoted by hirer·Actively reviewing applicants\nHow your profile and resume fit this job\nGet AI-powered advice on this job and more exclusive features with Premium.Reactivate Premium\nTailor my resume to this job Am I a good fit for this job?\nPeople you can reach out to\nTré and others in your network Show all\nMeet the hiring team\nGeorge Dix2ndBuilding Software Engineering teams for start-ups& scale-ups @ Few&FarJob poster · 4 mutual connections\nMessage\nAbout th

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answear the question based on the context below, if you can't answear the question
reply with 'I don't know'

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(question = "there is some question", context = "there is some context")

"\nAnswear the question based on the context below, if you can't answear the question\nreply with 'I don't know'\n\nContext: there is some context\n\nQuestion: there is some question\n\n"

In [7]:
chat = prompt | model | parser
chat.invoke({"context": "I am on England and weather is very bad", "question":"what country you currently at and how is the weather?"})

'I am currently in England and the weather is very bad. \n\n'

In [8]:
chat.input_schema.schema()

/tmp/ipykernel_68449/3594136560.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  chat.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embedding)

/home/salc2/workspace/rag-basic-system-study/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
retriever = vectorstore.as_retriever()

retriever.invoke("Passangers")

[Document(metadata={'producer': 'Skia/PDF m138', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36', 'creationdate': '2025-09-11T11:30:06+00:00', 'title': '(1) LinkedIn', 'moddate': '2025-09-11T11:30:06+00:00', 'source': 'jobspecs.pdf', 'total_pages': 4, 'page': 1, 'page_label': '2'}, page_content="More jobs\nSee less\nDriving best practices in backend engineering and mentoring juniordevelopers as the team grows.Collaborating with stakeholders to define technical roadmaps andarchitecture decisions.\nWhatʼs in it for You?\n💰  Salary: £90,000 - £110,000 + Equity📍  Hybrid work setup—London office (Kingʼs Cross) 1 day a week🚀  The chance to be part of a fast-growing startup making a real impactin healthcare📈  Career growth with the opportunity to transition into a leadership role\nApply if interested!\nSet alert for similar jobs\nBack End Developer, London Area, United Kingdom Off\nUnlock hiring insights on Few

In [11]:
from operator import itemgetter
chat = (
    {
        "context": itemgetter("question") | retriever , 
        "question": itemgetter("question")
    } | prompt | model | parser
)

In [ ]:
questions = [
    "what location is this job in?",
    "can this job be remotely",
    "what is the salary?",
    "what skills do we need to have?"
]

for question in questions:
    print(f"question: {question}")
    answ = chat.stream({"question": question})
    print(f"answear: {answ}")

question: what location is this job in?
answear: London Area, United Kingdom.
question: can this job be remotely
answear: Yes, the job is listed as "London Area, United Kingdom(Remote)". The job posting also mentions "Hybrid work setup—London office (King’s Cross) 1 day a week". 

Therefore, the job can be remotely.

question: what is the salary?
answear: £90,000 - £110,000 

question: what skills do we need to have?
answear: Python, FastAPI, AWS, Typescript, PostgreSQL, NumPy, Pandas.
